In [ ]:
import os

os.environ['IS_DEV_LOGGING'] = str(1)
os.environ['ENABLE_LOGGING'] = str(0)


PDF_PATH = Path("<REPLACE THIS WITH PATH TO PDF TO READ>") 
PAGE_PDF_START = 0
PAGE_PDF_END = 999
OUTPUT_WAV_PATH = f"<OUTPUT NAME HERE>.wav"
SPEAKER_WAV_FILES = [
    "<INPUT PATH TO SPEAKER WAV FILE>",
    "<INPUT PATH TO SPEAKER WAV FILE 2>",
    "...",
]

In [ ]:
import torch
import gc
import re

from wtpsplit import SaT
from tqdm.notebook import tqdm
from pathlib import Path
from TTS.api import TTS
from src.text_processors.str_to_sentences import split_into_sentences # so far better than nltk punkt
from src.parsers.pymupdf_parser import PyMuPDFParser
from src.text_processors.pre_processor import PreProcessor


In [ ]:
with PyMuPDFParser(PDF_PATH) as pdf_parser:
    pdf_parser.remove_all_links()
    result = pdf_parser.extract_text_between_pages(PAGE_PDF_START, PAGE_PDF_END)
    
result

### **CLEAN TEXT FROM UNWANTED CHARS AND SPLIT INTO SENTENCES** 

In [ ]:
pre_processor = PreProcessor()

res_clean = pre_processor.remove_unwanted_nonascii_chars(result)

res_clean = pre_processor.remove_extra_periods(res_clean)
res_clean = pre_processor.remove_leading_commas(res_clean)

res_clean

In [ ]:
sat = SaT("sat-12l-sm", style_or_domain="nl", language="nl") # TODO: leaks VRAM
sat.half().to("cuda")
res_clean = list(sat.split(res_clean))

# Move model to CPU
sat.model.model.cpu()
sat.model.cpu()
sat.cpu()

# Delete model
del sat.model.model 
del sat.model
del sat.tokenizer
del sat

torch.cuda.empty_cache()

# del sat
gc.collect()

res_clean

In [ ]:
dev = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{dev=}")

# TODO: https://docs.coqui.ai/en/dev/models/xtts.html --> check this site for more info about XTTS_v2
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to(dev)

In [ ]:
synth_dir = Path(OUTPUT_WAV_PATH.split(".")[0]+"_dir")
if not os.path.exists(synth_dir):
    os.mkdir(synth_dir)

In [ ]:
filename = OUTPUT_WAV_PATH.split(".")[0]
ext = "."+OUTPUT_WAV_PATH.split(".")[1]
audio_paths = []

for i, sent in enumerate(res_clean):
    audio_paths.append(synth_dir / Path(filename+f"_{i}"+ext))


for i, audio_path in enumerate(tqdm(audio_paths)):
    if not os.path.exists(audio_path):
        # Strip to remove added whitespaces by sentence splitter model. 
        # Then remove trailing punctuations in the hope that the XTTS model outputs better wav files
        res_clean[i] = pre_processor.remove_punctuations_at_end(res_clean[i].strip()) 
        
        tts.tts_to_file(
            text=res_clean[i], 
            speaker_wav=SPEAKER_WAV_FILES, 
            language="nl", 
            file_path=audio_path, 
            split_sentences=False
        )

In [ ]:
res_clean

In [ ]:
import wave

def concatenate_audio_wave(audio_clip_paths, output_path):
    """Concatenates several audio files into one audio file using Python's built-in wav module
    and save it to `output_path`. Note that extension (wav) must be added to `output_path`"""
    data = []
    for clip in audio_clip_paths:
        w = wave.open(clip, "rb")
        data.append([w.getparams(), w.readframes(w.getnframes())])
        w.close()
    output = wave.open(output_path, "wb")
    output.setparams(data[0][0])
    for i in range(len(data)):
        output.writeframes(data[i][1])
    output.close()

concatenate_audio_wave(list(map(str, audio_paths)), str(synth_dir / OUTPUT_WAV_PATH))

# remove audio sounds after merge
for audio_path in audio_paths:
    os.remove(audio_path)

In [ ]:
from IPython.display import Audio 
from IPython.core.display import display
def autoplay_audio():
    display(Audio(synth_dir / OUTPUT_WAV_PATH, autoplay=True))
autoplay_audio()